<a href="https://colab.research.google.com/github/mifm/examples/blob/master/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install numpy_financial

In [5]:
import numpy as np
import numpy_financial as npf
import scipy as sp

In [6]:
N = 5

C_t1 = np.asarray([100] + N * [0])
M_t = np.asarray(N * [5])
F_t = np.asarray(N * [1])
E_t = np.asarray(N * [10])
P_t = np.asarray(N * [2.97])
D_t = np.asarray(N * [0.0])

M_t1 = np.hstack([0, M_t])
F_t1 = np.hstack([0, F_t])
E_t1 = np.hstack([0, E_t])
P_t1 = np.hstack([0, P_t])
D_t1 = np.hstack([0, D_t])

r = 0.06
yr = np.arange(N+1)
depreciation_yr = [0, N]
depreciation = [0, 1]
inflation_yr = [0, N]
inflation = [0.03, 0.03]
ref_yr_inflation = 0
tax_rate=0.26
depre = np.interp(yr, depreciation_yr, depreciation)

infl = np.interp(yr, inflation_yr, inflation)

ind_ref = np.where(np.array(yr)==ref_yr_inflation)[0]
inflation_index = np.cumprod(1+np.array(infl))
inflation_index = inflation_index/inflation_index[ind_ref]

def get_NPV(P_t1):
    Net_revenue_t = E_t1 * P_t1

    EBITDA = (Net_revenue_t) * inflation_index#[1:]
    depreciation_on_each_year = np.hstack([0, np.diff(np.sum(C_t1)*depre)])

    EBIT = EBITDA - depreciation_on_each_year

    Taxes = EBIT*tax_rate
    Net_income = EBITDA - Taxes

    Cashflow = Net_income - C_t1 - M_t1 - F_t1 - D_t1

    NPV = npf.npv(r, Cashflow)

    LCOE = np.sum((C_t1 + M_t1 + F_t1 + D_t1) / (1 + r) ** yr) / np.sum(E_t1 / (1 + r) ** yr)
    return NPV, LCOE

def fun(price_el):
    NPV, _ = get_NPV(price_el)
    return NPV ** 2
out = sp.optimize.minimize(
    fun=fun,
    x0=50,
    method='SLSQP',
    tol=1e-10)



In [7]:
NPV, LCOE = get_NPV(P_t1)
break_even_price=out.x[0]


print('NPV: ', NPV)
print('LCOE: ', LCOE)
print('break_even_price: ', break_even_price)


NPV:  -2.465477215590031
LCOE:  2.9739640043118962
break_even_price:  3.042568347622598
